# Multi-Date Expected Proceeds Prediction Workflow (Local Version)

This notebook runs the expected proceeds prediction workflow for multiple inference dates and combines the results into a single DataFrame that gets pushed to Snowflake.

In [1]:
!pip3 install numpy==1.23.5
!pip3 install pandas==1.5.3
!pip3 install pyarrow==10.0.1
!pip3 install "snowflake-connector-python[pandas]"
!pip3 install snowflake-snowpark-python
!pip3 install tqdm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from tqdm import tqdm

# Import custom modules
from country_utils import add_signup_country_group
from data_utils import split_data_by_date, split_data_by_user_type
from trial_predictions import TrialPredictionModel
from direct_purchase_predictions import DirectPurchasePredictionModel
from lag_purchase_predictions import LagPurchasePredictionModel

## Connect to Snowflake

We'll use the configuration from our `config.py` file to connect to Snowflake.

In [3]:
# Import Snowflake connection config
import config

# Snowflake connection
from snowflake.snowpark import Session

def get_snowflake_session():
    """Create and return a Snowflake session"""
    connection_parameters = {
        "account": config.SNOWFLAKE_ACCOUNT,
        "user": config.SNOWFLAKE_USER,
        "role": config.SNOWFLAKE_ROLE,
        "warehouse": config.SNOWFLAKE_WAREHOUSE,
        "database": config.SNOWFLAKE_DATABASE,
        "schema": config.SNOWFLAKE_SCHEMA,
        "authenticator": config.SNOWFLAKE_AUTHENTICATOR
    }
    
    session = Session.builder.configs(connection_parameters).create()
    print(f"Connected to Snowflake as {config.SNOWFLAKE_USER}")
    return session

# Create a Snowflake session
session = get_snowflake_session()

2025-03-18 19:07:38,860 - snowflake.connector.connection - INFO - Snowflake Connector for Python Version: 3.14.0, Python Version: 3.9.6, Platform: macOS-15.2-arm64-arm-64bit
2025-03-18 19:07:38,860 - snowflake.connector.connection - INFO - Connecting to GLOBAL Snowflake domain
2025-03-18 19:07:38,861 - snowflake.connector.connection - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=64537&proof_key=%2BG6D3kg%2Brzw2bLvo1JjeVoL%2F8Q6hCQb9XRaRAwocKXI%3D to authenticate...


2025-03-18 19:07:45,481 - snowflake.snowpark.session - INFO - Snowpark Session information: 
"version" : 1.29.1,
"python.version" : 3.9.6,
"python.connector.version" : 3.14.0,
"python.connector.session.id" : 9893985661975982,
"os.name" : Darwin



Connected to Snowflake as meri-kris.jaama@go1.com


## 1. Define Inference Dates

Specify the dates for which you want to run the workflow.

In [4]:
# Define inference dates
# Option 1: Manually specify dates
inference_dates = [
    '2025-01-01',
    '2024-12-01',
    '2024-12-02'
]

# Option 2: Generate a range of dates
start_date = '2024-1-01'
end_date = '2025-01-31'
start = datetime.strptime(start_date, '%Y-%m-%d')
end = datetime.strptime(end_date, '%Y-%m-%d')
inference_dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end - start).days + 1)]

print(f"Running workflow for {len(inference_dates)} dates: {inference_dates}")

Running workflow for 62 dates: ['2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04', '2024-12-05', '2024-12-06', '2024-12-07', '2024-12-08', '2024-12-09', '2024-12-10', '2024-12-11', '2024-12-12', '2024-12-13', '2024-12-14', '2024-12-15', '2024-12-16', '2024-12-17', '2024-12-18', '2024-12-19', '2024-12-20', '2024-12-21', '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25', '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29', '2024-12-30', '2024-12-31', '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04', '2025-01-05', '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-11', '2025-01-12', '2025-01-13', '2025-01-14', '2025-01-15', '2025-01-16', '2025-01-17', '2025-01-18', '2025-01-19', '2025-01-20', '2025-01-21', '2025-01-22', '2025-01-23', '2025-01-24', '2025-01-25', '2025-01-26', '2025-01-27', '2025-01-28', '2025-01-29', '2025-01-30', '2025-01-31']


## 2. Load Data

Load the necessary data from Snowflake.

In [5]:
# Fetch data from Snowflake
print("Loading data from Snowflake...")
input_query = """
    SELECT 
        *
    FROM blinkist_dev.dbt_mjaama.exp_proceeds_input
    """
    
input_df = session.sql(input_query).to_pandas()

product_query = """
    select sku as product_name, price 
    from BLINKIST_PRODUCTION.reference_tables.product_dim
    where is_purchasable;
    """
    
product_df = session.sql(product_query).to_pandas()

print(f"Loaded {len(input_df)} input records and {len(product_df)} product records")

# Convert column names to lowercase for compatibility with utility functions
input_df.columns = input_df.columns.str.lower()
print("Column names converted to lowercase")

Loading data from Snowflake...
Loaded 8060581 input records and 833 product records
Column names converted to lowercase


## 3. Add Country Groups

Add country groups to the input data to help with analysis.

In [6]:
# Add country groups
print("Adding country groups...")
input_df = add_signup_country_group(input_df)
print("Country groups added")

Adding country groups...


/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:37: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  us_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (temp_df.signup_country == "US") & (
/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  row_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (


Country groups added


## 4. Define Workflow Function

Define a function to run the workflow for a single inference date.

In [7]:
def run_workflow_for_date(input_df, product_df, inference_date, training_window_days=180):
    """Run the workflow for a single inference date"""
    print(f"\nProcessing inference date: {inference_date}")
    
    # 1. Split data by date
    inference_df, training_d8_df, training_d100_df = split_data_by_date(
        input_df,
        inference_date=inference_date,
        training_window_days=training_window_days,
        date_column='report_date'
    )
    
    print(f"Inference data: {len(inference_df)} records")
    print(f"Training d8 data: {len(training_d8_df)} records")
    print(f"Training d100 data: {len(training_d100_df)} records")
    
    # Check if we have enough inference data
    if len(inference_df) == 0:
        print(f"WARNING: No inference data found for {inference_date}. Skipping.")
        return None
    
    # 2. Split data by user type
    trial_inference, day0_payers_inference, other_inference = split_data_by_user_type(inference_df)
    trial_training_d8, day0_payers_training_d8, other_training_d8 = split_data_by_user_type(training_d8_df)
    trial_training_d100, day0_payers_training_d100, other_training_d100 = split_data_by_user_type(training_d100_df)
    
    print(f"Trial users: {len(trial_inference)} inference, {len(trial_training_d8)} training d8, {len(trial_training_d100)} training d100")
    print(f"Day 0 payers: {len(day0_payers_inference)} inference, {len(day0_payers_training_d8)} training d8, {len(day0_payers_training_d100)} training d100")
    print(f"Other users: {len(other_inference)} inference, {len(other_training_d8)} training d8, {len(other_training_d100)} training d100")
    
    # 3. Train models
    print("Training models...")
    
    print("Train trial model")
    # Trial model
    trial_model = TrialPredictionModel(product_dim_df=product_df)
    trial_model.fit(trial_training_d8, trial_training_d100)
    
    print("Train direct purchase model")
    # Direct purchase model
    direct_model = DirectPurchasePredictionModel()
    direct_model.fit(day0_payers_training_d100)
    
    print("Train lag purchase model")
    # Lag purchase model
    lag_model = LagPurchasePredictionModel(product_dim_df=product_df)
    lag_model.fit(other_training_d8, other_training_d100)
    
    # 4. Make predictions
    print("Making predictions...")
    predictions = []
    
    if not trial_inference.empty:
        trial_predictions = trial_model.predict(trial_inference)
        trial_predictions['user_type'] = 'trial'
        predictions.append(trial_predictions)
        print(f"Generated {len(trial_predictions)} predictions for trial users")
    
    if not day0_payers_inference.empty:
        direct_predictions = direct_model.predict(day0_payers_inference)
        direct_predictions['user_type'] = 'day0_payer'
        predictions.append(direct_predictions)
        print(f"Generated {len(direct_predictions)} predictions for day 0 payers")
    
    if not other_inference.empty:
        lag_predictions = lag_model.predict(other_inference)
        lag_predictions['user_type'] = 'other'
        predictions.append(lag_predictions)
        print(f"Generated {len(lag_predictions)} predictions for other users")
    
    # Check if we have any predictions
    if not predictions:
        print("No predictions generated - inference data may be empty")
        return None
    
    # Combine predictions
    all_predictions = pd.concat(predictions, ignore_index=True)
    
    # Add inference date as a column
    all_predictions['inference_date'] = inference_date
    
    print(f"Total predictions: {len(all_predictions)}")
    return all_predictions

## 5. Run Workflow for All Dates

Run the workflow for each inference date and collect the results.

In [8]:
# Run workflow for all dates
all_results = []

# Use standard tqdm instead of the notebook version
for date in tqdm(inference_dates, desc="Processing dates"):
    try:
        result = run_workflow_for_date(input_df, product_df, date)
        if result is not None:
            all_results.append(result)
    except Exception as e:
        print(f"Error processing date {date}: {str(e)}")

# Combine all results
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    print(f"\nCombined results: {len(combined_results)} records across {len(all_results)} dates")
else:
    print("No results to combine")

Processing dates:   0%|          | 0/62 [00:00<?, ?it/s]


Processing inference date: 2024-12-01
Inference data: 10398 records
Training d8 data: 1524968 records
Training d100 data: 1228202 records
Trial users: 290 inference, 85090 training d8, 110275 training d100
Day 0 payers: 1766 inference, 50951 training d8, 26998 training d100
Other users: 8326 inference, 1388151 training d8, 1090691 training d100
Training models...
Train trial model
Starting trial model fitting...
Calculating historical proceeds...
Found 24948 records with non-zero proceeds
Global average proceeds: 56.1808405867729
Calculated average proceeds for 68 products
Calculated average proceeds for 105 product-country combinations
Target values before outlier removal: [0 1]
Target distribution: 0s: 60142, 1s: 24948
Removed 0 outliers from 85090 samples (0.00%)
Numerical features: ['eur_marketing_spend', 'impressions', 'clicks']
Categorical features: ['channel_group', 'marketing_network_id', 'target_market', 'signup_country_group', 'signup_client_platform', 'plan_tier']
Fitting p

Processing dates:   2%|▏         | 1/62 [05:01<5:06:38, 301.62s/it]


Processing inference date: 2024-12-02
Inference data: 9856 records
Training d8 data: 1526960 records
Training d100 data: 1229944 records
Trial users: 252 inference, 84714 training d8, 110011 training d100
Day 0 payers: 1780 inference, 51352 training d8, 27045 training d100
Other users: 7815 inference, 1390119 training d8, 1092645 training d100
Training models...
Train trial model
Starting trial model fitting...
Calculating historical proceeds...
Found 24796 records with non-zero proceeds
Global average proceeds: 56.25669739517142
Calculated average proceeds for 68 products
Calculated average proceeds for 106 product-country combinations
Target values before outlier removal: [0 1]
Target distribution: 0s: 59918, 1s: 24796
Removed 0 outliers from 84714 samples (0.00%)
Numerical features: ['eur_marketing_spend', 'impressions', 'clicks']
Categorical features: ['channel_group', 'marketing_network_id', 'target_market', 'signup_country_group', 'signup_client_platform', 'plan_tier']
Fitting p

Processing dates:   3%|▎         | 2/62 [10:10<5:05:45, 305.76s/it]

...


Processing inference date: 2025-01-30
Inference data: 8964 records
Training d8 data: 1812454 records
Training d100 data: 1373409 records
Trial users: 242 inference, 68828 training d8, 90641 training d100
Day 0 payers: 269 inference, 85651 training d8, 38769 training d100
Other users: 8443 inference, 1656873 training d8, 1243412 training d100
Training models...
Train trial model
Starting trial model fitting...
Calculating historical proceeds...
Found 16433 records with non-zero proceeds
Global average proceeds: 62.817657515099285
Calculated average proceeds for 72 products
Calculated average proceeds for 95 product-country combinations
Target values before outlier removal: [0 1]
Target distribution: 0s: 52395, 1s: 16433
Removed 0 outliers from 68828 samples (0.00%)
Numerical features: ['eur_marketing_spend', 'impressions', 'clicks']
Categorical features: ['channel_group', 'marketing_network_id', 'target_market', 'signup_country_group', 'signup_client_platform', 'plan_tier']
Fitting pre

Processing dates:  98%|█████████▊| 61/62 [5:42:06<05:53, 353.46s/it]


Processing inference date: 2025-01-31
Inference data: 9272 records
Training d8 data: 1811920 records
Training d100 data: 1379328 records
Trial users: 280 inference, 68501 training d8, 90527 training d100
Day 0 payers: 248 inference, 85845 training d8, 39006 training d100
Other users: 8739 inference, 1656468 training d8, 1249199 training d100
Training models...
Train trial model
Starting trial model fitting...
Calculating historical proceeds...
Found 16309 records with non-zero proceeds
Global average proceeds: 62.937066821183656
Calculated average proceeds for 72 products
Calculated average proceeds for 94 product-country combinations
Target values before outlier removal: [0 1]
Target distribution: 0s: 52192, 1s: 16309
Removed 0 outliers from 68501 samples (0.00%)
Numerical features: ['eur_marketing_spend', 'impressions', 'clicks']
Categorical features: ['channel_group', 'marketing_network_id', 'target_market', 'signup_country_group', 'signup_client_platform', 'plan_tier']
Fitting pre

Processing dates: 100%|██████████| 62/62 [5:47:52<00:00, 336.65s/it]


Combined results: 707467 records across 62 dates


## 6. Save Results

Save the combined results locally and to Snowflake.

In [9]:

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Save to CSV
date_range = f"{inference_dates[0]}_to_{inference_dates[-1]}"
output_path = f"output/predictions_{date_range}.csv"
combined_results.to_csv(output_path, index=False)
print(f"Predictions saved locally to {output_path}")

# Save to Snowflake
print("\nSaving results to Snowflake...")
# Reconnect to Snowflake
session = get_snowflake_session()


# Convert to Snowpark DataFrame
snowpark_df = session.create_dataframe(combined_results)

# Save to Snowflake table
table_name = "BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250319"

# Append to existing table or create new one
snowpark_df.write.mode("append").save_as_table(table_name)

print(f"Predictions for {len(inference_dates)} dates saved to {table_name}")


2025-03-19 01:00:13,956 - snowflake.connector.connection - INFO - Snowflake Connector for Python Version: 3.14.0, Python Version: 3.9.6, Platform: macOS-15.2-arm64-arm-64bit
2025-03-19 01:00:13,957 - snowflake.connector.connection - INFO - Connecting to GLOBAL Snowflake domain
2025-03-19 01:00:13,957 - snowflake.connector.connection - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Predictions saved locally to output/predictions_2024-12-01_to_2025-01-31.csv

Saving results to Snowflake...
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=52619&proof_key=zTvIpLRx%2F%2BcvJvPBwcRfSHmyC08FKBDG%2BRTYUCG1VMI%3D to authenticate...


2025-03-19 04:27:57,876 - snowflake.snowpark.session - INFO - Snowpark Session information: 
"version" : 1.29.1,
"python.version" : 3.9.6,
"python.connector.version" : 3.14.0,
"python.connector.session.id" : 9893985662164914,
"os.name" : Darwin



Connected to Snowflake as meri-kris.jaama@go1.com
Predictions for 62 dates saved to BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250319


## 7. Close Snowflake Session

Close the Snowflake session when done.

In [10]:
# Close Snowflake session
session.close()
print("Snowflake session closed")

2025-03-19 04:28:40,894 - snowflake.snowpark.session - INFO - Closing session: 9893985662164914
2025-03-19 04:28:40,895 - snowflake.snowpark.session - INFO - Canceling all running queries
2025-03-19 04:28:41,110 - snowflake.connector.connection - INFO - closed
2025-03-19 04:28:41,250 - snowflake.connector.connection - INFO - No async queries seem to be running, deleting session
2025-03-19 04:28:41,427 - snowflake.snowpark.session - INFO - Closed session: 9893985662164914


Snowflake session closed


In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
from snowflake.snowpark import Session
import config
from prediction_processor import PredictionProcessor

# Connect to Snowflake
def get_snowflake_session():
    connection_parameters = {
        "account": config.SNOWFLAKE_ACCOUNT,
        "user": config.SNOWFLAKE_USER,
        "role": config.SNOWFLAKE_ROLE,
        "warehouse": config.SNOWFLAKE_WAREHOUSE,
        "database": config.SNOWFLAKE_DATABASE,
        "schema": config.SNOWFLAKE_SCHEMA,
        "authenticator": config.SNOWFLAKE_AUTHENTICATOR
    }
    session = Session.builder.configs(connection_parameters).create()
    print(f"Connected to Snowflake as {config.SNOWFLAKE_USER}")
    return session

# Initialize Snowflake session
session = get_snowflake_session()

# Define source and target table names
source_table = "BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250319"
target_table = "BLINKIST_DEV.DBT_MJAAMA.AGGREGATED_EXPECTED_PROCEEDS_20250320"

# Read data from Snowflake
print(f"Reading data from {source_table}...")
query = f"SELECT * FROM {source_table}"
predictions_df = session.sql(query).to_pandas()
print(f"Read {len(predictions_df)} rows from source table")

# Initialize the PredictionProcessor with the fixed implementation
processor = PredictionProcessor()

# Process the predictions
print("Processing predictions...")
aggregated_df = processor.process_predictions(predictions_df)
print(f"Processed data into {len(aggregated_df)} aggregated rows")

# Check for NaN values in the calculated columns
roi_columns = ['roi_d8', 'exp_roi_d8', 'roi_d100', 'exp_roi_d100', 
               'actual_roi_d8', 'actual_roi_d100', 'cpa']
nan_counts = {col: aggregated_df[col].isna().sum() for col in roi_columns if col in aggregated_df.columns}

print("\nNaN value check:")
for col, count in nan_counts.items():
    print(f"{col}: {count} NaN values out of {len(aggregated_df)} rows ({count/len(aggregated_df)*100:.2f}%)")

if any(count > 0 for count in nan_counts.values()):
    print("\nWARNING: NaN values detected in calculated columns. Review the data before proceeding.")
    # Optional: Uncomment to stop execution if NaN values are found
    # raise ValueError("NaN values detected in calculated columns")
else:
    print("\nNo NaN values found in calculated columns. Proceeding with Snowflake upload.")

# Display sample of the aggregated data
print("\nSample of aggregated data:")
display(aggregated_df.head())

# Write the aggregated data back to Snowflake
print(f"\nWriting aggregated data to {target_table}...")

# Check if the target table exists
table_exists = session.sql(f"SHOW TABLES LIKE '{target_table}'").collect()

if table_exists:
    print(f"Target table {target_table} exists. It will be overwritten.")
    # Drop the existing table
    session.sql(f"DROP TABLE IF EXISTS {target_table}").collect()
    print(f"Existing table {target_table} dropped.")

# Convert pandas DataFrame to Snowpark DataFrame
aggregated_snowpark_df = session.create_dataframe(aggregated_df)

# Write to Snowflake with overwrite mode
aggregated_snowpark_df.write.mode("overwrite").save_as_table(target_table)

print(f"Successfully wrote {len(aggregated_df)} rows to {target_table}")

# Close Snowflake session
session.close()
print("Snowflake session closed")

2025-03-19 09:07:27,287 - snowflake.connector.connection - INFO - Snowflake Connector for Python Version: 3.14.0, Python Version: 3.9.6, Platform: macOS-15.2-arm64-arm-64bit
2025-03-19 09:07:27,288 - snowflake.connector.connection - INFO - Connecting to GLOBAL Snowflake domain
2025-03-19 09:07:27,289 - snowflake.connector.connection - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=56700&proof_key=%2F2jbKVPBsfLxbTYRj2Fd6rTZ7TvZiMs3F79%2FaKe%2B4t0%3D to authenticate...


2025-03-19 09:07:33,854 - snowflake.snowpark.session - INFO - Snowpark Session information: 
"version" : 1.29.1,
"python.version" : 3.9.6,
"python.connector.version" : 3.14.0,
"python.connector.session.id" : 9893985662438446,
"os.name" : Darwin



Connected to Snowflake as meri-kris.jaama@go1.com
Reading data from BLINKIST_DEV.DBT_MJAAMA.MULTI_DATE_EXPECTED_PROCEEDS_20250319...
Read 1076388 rows from source table
Processing predictions...
Processed data into 4817 aggregated rows

NaN value check:
exp_roi_d8: 0 NaN values out of 4817 rows (0.00%)
exp_roi_d100: 0 NaN values out of 4817 rows (0.00%)
actual_roi_d8: 0 NaN values out of 4817 rows (0.00%)
actual_roi_d100: 0 NaN values out of 4817 rows (0.00%)
cpa: 0 NaN values out of 4817 rows (0.00%)

No NaN values found in calculated columns. Proceeding with Snowflake upload.

Sample of aggregated data:


,report_date,channel_group,marketing_network_id,account_id,campaign_name,campaign_id,adgroup_name,adgroup_id,target_market,total_users,...,eur_proceeds_d0,eur_proceeds_d8,eur_proceeds_d100,expected_proceeds_d8,expected_proceeds_d100,exp_roi_d8,exp_roi_d100,actual_roi_d8,actual_roi_d100,cpa
0,2024-12-01,paid_content,facebook,927736373941355,20240705_FB_PRO_ES_Web2App_iOS_S2S_MAN_MNC_Sub...,6595671883259,20240705_FB_PRO_ES_Web2App_iOS_S2S_MAN_MNC_Sub...,6595671882659,row,1,...,0.000000,0.000000,0.000000,0.330357,0.511162,0.000000,0.000000,0.000000,0.000000,0.00
1,2024-12-01,paid_content,facebook,927736373941355,20240705_FB_PRO_ES_Web2App_iOS_S2S_MAN_MNC_Sub...,6595671883259,20240815_FB_PRO_ES_Web2App_ALL_S2S_MAN_MNC_Sub...,6602384189259,row,4,...,103.332614,103.332614,103.332614,103.400599,103.400599,0.000000,0.000000,0.000000,0.000000,0.00
2,2024-12-01,paid_content,facebook,927736373941355,20240705_FB_PRO_ES_Web2App_iOS_S2S_MAN_MNC_Sub...,6595671883259,20241014_FB_PRO_ES_Web2App_ALL_S2S_MAN_MNC_Sub...,6614643557859,row,1,...,0.000000,0.000000,0.000000,0.067985,0.067985,0.000000,0.000000,0.000000,0.000000,0.00
3,2024-12-01,paid_content,facebook,927736373941355,20240712_FB_PRO_T1_Web2App_ALL_S2S_MAN_MNC_Sub...,6596549145459,20240712_FB_PRO_T1_Web2App_ALL_S2S_MAN_MNC_Sub...,6596549145259,row,1,...,0.000000,0.000000,0.000000,0.143144,0.229921,0.000000,0.000000,0.000000,0.000000,0.00
4,2024-12-01,paid_content,facebook,927736373941355,20240712_FB_PRO_T1_Web2App_ALL_S2S_MAN_MNC_Sub...,6596549145459,20241121_FB_PRO_T1_Web2App_ALL_S2S_MAN_MNC_Sub...,6622384529259,row,3,...,25.268073,25.268073,25.268073,25.781309,26.405292,0.076004,0.077843,0.074491,0.074491,113.07



Writing aggregated data to BLINKIST_DEV.DBT_MJAAMA.AGGREGATED_EXPECTED_PROCEEDS_20250320...


2025-03-19 09:07:59,812 - snowflake.snowpark.session - INFO - Closing session: 9893985662438446
2025-03-19 09:07:59,813 - snowflake.snowpark.session - INFO - Canceling all running queries


Successfully wrote 4817 rows to BLINKIST_DEV.DBT_MJAAMA.AGGREGATED_EXPECTED_PROCEEDS_20250320


2025-03-19 09:08:00,043 - snowflake.connector.connection - INFO - closed
2025-03-19 09:08:00,176 - snowflake.connector.connection - INFO - No async queries seem to be running, deleting session
2025-03-19 09:08:00,363 - snowflake.snowpark.session - INFO - Closed session: 9893985662438446


Snowflake session closed
